 * Targets
        https://github.com/NVIDIA/cuda-quantum

# CUDA Quantum Tutorial #2
  
 
    * Kernel builder
        https://nvidia.github.io/cuda-quantum/latest/using/cudaq/builder.html
  
    * Simulation backends
        https://nvidia.github.io/cuda-quantum/latest/using/simulators.html

    * Variational algorithms
        https://nvidia.github.io/cuda-quantum/latest/using/cudaq/variational.html
        

    Outline 

    1. CUDA Quantum Kernels continued
        1.1 Adjoint 
        1.2 Conditionals 

    2. Variational quantum algorithms (VQAs)
        2.1 General
        2.2 VQA in CUDA Q
            2.2.1 Using cudaq::observe and optimize explicitly
            2.2.2 using cudaq.vqe()

    3. Targets continued
        3.1 Hardware backends
            3.1.1 Quantum hadrware integration
        3.2 Quantum Circuit Simulation
            3.2.1 cuQuantum Simulator backends
                 - Statevector (custatevec)
                 - Tensornet (cutensornet)
            3.2.2 Density-matrix (dm)
            
    4. Noise modeling 
        4.1 Background 
        4.2 With density-matrix (dm) backend

    

### 3. CUDA Quantum `target` 
A `target` is a specification of the desired platform and simulator / QPU. It can be specified as a runtime flag in Python. Alteratively, it can also be specified within the application code. 

     Simulation backends
    - state-vector (`cuStateVec`) 
    - tensor-network (`cuTensorNet`)
    - density-matrix (`dm`) 

     Hardware support
    - CPU only, multi-threaded   
    - Single GPU   
    - Multi-GPU 
    - Multi-QPU 
    - Multi-node
    - QPU 

In [1]:
### 5. Power of GPU acceleration 101

In [3]:
# Print all the available targets on your machine
import cudaq

targets = cudaq.get_targets()

for t in targets:
     print(t)

Target orca
	simulator=qpp
	platform=default
	description=

Target quantinuum
	simulator=qpp
	platform=default
	description=

Target oqc
	simulator=qpp
	platform=default
	description=

Target nvidia-fp64
	simulator=custatevec_fp64
	platform=default
	description=The NVIDIA FP64 Target provides a simulated QPU via single-GPU cuStateVec integration on FP64 types.

Target nvidia
	simulator=custatevec_fp32
	platform=default
	description=The NVIDIA Target provides a simulated QPU via single-GPU cuStateVec integration on FP32 types.

Target nvidia-mqpu
	simulator=custatevec_fp32
	platform=mqpu
	description=The NVIDIA MQPU Target provides a simulated QPU for every available CUDA GPU on the underlying system. Each QPU is simulated via cuStateVec FP32. This target enables asynchronous parallel execution of quantum kernel tasks.

Target default
	simulator=qpp
	platform=default
	description=Default OpenMP CPU-only simulated QPU.

Target nvidia-mqpu-fp64
	simulator=custatevec_fp64
	platform=mqpu
	d

### 3.2.1. Simulator backends - cuQuantum
    CUDA Quantum's workhorse for quantum circuit simulation. It is a high performance library containing the following two types of simulators.

##### Statevector simulator    
    * State vector simulators serve as the main vehicle for circuit simulations. 
    * They maintain an accurate representation of the quantum state, known as the state vector, throughout the simulation. 
    * Each gate that is applied corresponds to a matrix-vector multiplication.

##### Tensornet simulator 
    * The tensor network method is a technique that represents the quantum state of N qubits as a series of tensor contractions.
    * The main challenge is to compute these tensor contractions efficiently. 
    * It can handle a large number of qubits for short circut depths.

    Note: To run with the cutensornet target, you will need to pull the CUDA Quantum docker image with the tag latest-hpc.

    Make sure that that your Docker container was launched with the `--gpus all` flag turned on.

#### 3.2.2 Density matrix simulator
    * Simulates quantum circuits under the influence of noise. 
    * Currently, it calls the QPP library under the hood and has only CPU support.

    To discuss the density matrix simulator further, we need to introduce a couple of new concepts.

    Bell State 
 $\dfrac{1}{\sqrt{2}}(\ket{00} +\ket{11})$, maximally entangled 2-qubit state.

    GHZ state
$\dfrac{1}{\sqrt{2}}(\ket{0}^{\otimes N} +\ket{1}^{\otimes N})$, maximally entangled N-qubit state, $N \ge 3$.

In [2]:
# To run with available gpu using a command line flag:
# python3 ghz_state.py --target nvidia
# To run with cpu (very slow) using a command line flag:
# python3 ghz_state.py


import cudaq

cudaq.set_target("nvidia") # change target to "default" for cpu

def ghz_state(N):

    kernel = cudaq.make_kernel()
    q = kernel.qalloc(N)

    kernel.h(q[0])
    for i in range(N - 1):
      kernel.cx(q[i], q[i + 1])

    kernel.mz(q)
    return kernel

n = 30
print("Preparing GHZ state for", n, "qubits.")
kernel = ghz_state(n)
counts = cudaq.sample(kernel)
counts.dump()

Preparing GHZ state for 30 qubits.
{ 000000000000000000000000000000:521 111111111111111111111111111111:479 }
